# Quantitative Risk Management

Click <a href="https://colab.research.google.com/github/Lolillosky/QuantRiskManagement/blob/main/NOTEBOOKS/11_PCA_IR.ipynb">
    <img src="https://upload.wikimedia.org/wikipedia/commons/d/d0/Google_Colaboratory_SVG_Logo.svg" width="30" alt="Google Colab">
</a> to open this notebook in Google Colab.


In this exercise, you are going to perform principal component analysis to the Euro Swap Curve.

## Import main libraries:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn
from IPython.display import clear_output

## Import Libraries from Github Repository 

In [21]:
import sys
#sys.path.append('../CODE')  # Adjust the path as necessary


from IPython.display import clear_output

!rm -r {'QuantRiskManagement'}

!git clone https://github.com/Lolillosky/QuantRiskManagement.git

import sys
sys.modules.pop
sys.path.insert(0,'QuantRiskManagement/CODE')

clear_output()


In [ ]:
PATH_DATA = '../DATA/'
#PATH_DATA = '/content/QuantRiskManagement/DATA/'

PATH_HTML = '../docs/'
PATH_FIGS = '../FIGS/'


## Parse historical swap data:

In [ ]:
swap_rates = pd.read_csv(PATH_DATA + 'IRS_History.csv', index_col=0, parse_dates=True, date_format='%d/%m/%Y')

swap_rates /= 100

swap_rates = swap_rates.loc['2020':]

matutities = [float(x) for x in swap_rates.columns]

swap_rates.head()


Let's explore how data looks like:

In [ ]:
import plotly.graph_objects as go


fig = go.Figure(data=[go.Surface(x=swap_rates.columns,y= swap_rates.index,z=swap_rates.values)])

fig.update_layout(
    scene=dict(
        xaxis_title='Matutities',  
        yaxis_title='Dates', 
        zaxis_title='Swap Rates'))

#fig.write_html(PATH_HTML + 'swap_rates.html')

fig.show()


In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 2**128

f, ax = plt.subplots(1)

series, = ax.plot([], '-o', markersize=3)

num_frames = len(swap_rates)


ax.set_xlim(0,matutities[-1])
ax.set_ylim(swap_rates.min().min(), swap_rates.max().max())

def animate(k):

    series.set_data(matutities,swap_rates.iloc[k])
    
    ax.set_title(swap_rates.index[k].strftime('%d/%m/%Y'))



ani = FuncAnimation(f, animate, frames=num_frames, interval = 20)

plt.show()

In [ ]:
html_str = ani.to_jshtml()

# with open(PATH_HTML + 'Film.html', 'w') as f:
#    f.write(html_str)

HTML(html_str)


Swap rates of different maturities seem to be correlated. Let's compute the correlation matrix of daily increments:

In [ ]:
swap_rates_inc = swap_rates.diff().dropna()

corr_swap = swap_rates_inc.corr()



In [ ]:
plt.figure(figsize=(10, 8))

sns.heatmap(corr_swap, annot=False, fmt=".2f", cmap='coolwarm', cbar=True, square=True)

#plt.savefig(PATH_FIGS + 'correlation_matrix.pdf', dpi=300, bbox_inches='tight')

Swap rate increments seem to be highly correlated. Correlation increases for closer tenors.

Let us run a PCA on swap rate increments. In order to do so, we will use a Scikit-Learn pipeline with 2 steps: 

* A scaler that normalizes the data (substracting mean and dividing by standard deviation of each risk factor).
* A PCA model.

In [ ]:
# We import the necessary tools
from sklearn.decomposition import PCA # PCA analysis
from sklearn.pipeline import Pipeline # Pipeline
from sklearn.preprocessing import StandardScaler # Scaler

# We define a pipeline
PCA_pipeline = Pipeline([('Scaler', StandardScaler()), ('PCA', PCA())])

# We fit the PCA model
PCA_pipeline.fit(swap_rates_inc);


Let us explore the value of the eigenvalues and eigenvectors.

In [ ]:
# To explore the model results we access the PCA step
# of the pipeline model
PCA_model = PCA_pipeline.named_steps['PCA']

f, ax = plt.subplots(3,1)

# We plot the eigenvalues
ax[0].plot(matutities, PCA_model.explained_variance_, 'o-')
ax[0].set_title('Eigenvalues')

# We plot the cummulative sun of the eigenvalues as 
# a percentage over the sum of them
ax[1].plot(matutities, np.cumsum(PCA_model.explained_variance_)/np.sum(PCA_model.explained_variance_), 'o-')
ax[1].set_title('Cum Eigenvalues variance %')

ax[2].plot(matutities, PCA_model.components_[0], 'o-', label = '1st eigenvector')
ax[2].plot(matutities, PCA_model.components_[1], 'o-', label = '2st eigenvector')
ax[2].plot(matutities, PCA_model.components_[2], 'o-', label = '3rd eigenvector')

ax[2].legend()

ax[2].set_title('Ppal components shape')

ax[2].axhline(0,color = 'grey', linestyle = '--')
f.set_size_inches(5,12)

Notice that the eigenvalues are variances of independent variables. To that extend, their cummulative sum has some meaning. Notive that 3 principal components accumulate more than 98% of the variance.

Also notice that the shapes of the eigenvectors also have some meanings:

* The first principal component represents a roughly parallel movement.
* The second principal component represents a movement where short term and long term maturities are shocked with opposite sings (steepening / flattening) or the IR curve.
* The third principal component represents shocks where the intermediate part of the curve moves in opposite direction to the short and long ends.

The reduction allows us to represent curve movements in lower dimensions.

In [ ]:
dates_good_format = [d.strftime('%d, %b %Y') for d in swap_rates_inc.index]

pca_realization = PCA_pipeline.transform(swap_rates_inc)

fig = go.Figure(data=go.Scatter(x=pca_realization[:,0],
                                y=pca_realization[:,1],
                                mode='markers',
                                #marker_color=data['Population'],
                                text=dates_good_format)) # hover text goes here

fig.update_layout(title='IR PCA')
fig.show()

In [ ]:
window = 260

p_comp_len = len(swap_rates_inc) - window

p_comp1 = pd.DataFrame(data = np.zeros((p_comp_len, swap_rates_inc.shape[1])), \
                      index = swap_rates_inc.index[window:])

p_comp2 = pd.DataFrame(data = np.zeros((p_comp_len, swap_rates_inc.shape[1])), \
                      index = swap_rates_inc.index[window:])

p_comp3 = pd.DataFrame(data = np.zeros((p_comp_len, swap_rates_inc.shape[1])), \
                      index = swap_rates_inc.index[window:])


for i in range(window, len(swap_rates_inc)):

  PCA_pipeline.fit(swap_rates_inc.iloc[i-window:i])

  c1 = PCA_pipeline.named_steps['PCA'].components_[0]
  c2 = PCA_pipeline.named_steps['PCA'].components_[1]
  c3 = PCA_pipeline.named_steps['PCA'].components_[2]

  if c1[0] <0:
    c1*=-1
  if c2[0] <0:
    c2*=-1
  if c3[0] <0:
    c3*=-1

  p_comp1.iloc[i-window] = c1
  p_comp2.iloc[i-window] = c2
  p_comp3.iloc[i-window] = c3

  if (i-window)%300 == 0:
    clear_output()
    print(str(i-window) + '/' + str(len(swap_rates_inc)-window))



The eigenvectors shape is quite stable throughout time.

In [ ]:
def plot_curves_date_animate(date, curves, ax, series, min_maturity, max_maturity,
                            min_yield, max_yield, title):

  count = 0
  
  for k, v in curves.items():
  
    x = np.array(v[1])
      
    y = v[0].loc[date].values[np.logical_and(x <= max_maturity, x >= min_maturity)]
    
    x = x[np.logical_and(x <= max_maturity, x >= min_maturity)]
  
    series[count].set_data(x, y)
    
    count+=1
    
  ax.set_ylim(min_yield,max_yield)
  ax.set_xlim(min_maturity,max_maturity)
    
  ax.set_title(date.strftime(title + '%d, %b %Y'))
    
  ax.legend(loc='upper left')

def swap_curve_animation(curves, date_range, fps, min_maturity, max_maturity, 
                         min_yield, max_yield, title):
  
  f, ax = plt.subplots(1)

  series = []

  for k, v in curves.items():

    serie, = ax.plot([], '.-', label = k)

    series += [serie]

  def animate(k):

      date = list(curves.values())[0][0].index[k]

      plot_curves_date_animate(date, curves, ax, series, min_maturity, max_maturity,
                              min_yield, max_yield, title)



  ani = FuncAnimation(f, animate, frames=date_range, interval = fps)

  plt.show()
  
  return ani  

In [ ]:
curves = {'1st ppal comp' : (p_comp1, matutities),
           '2nd ppal comp' : (p_comp2, matutities),
           '3rd ppal comp' : (p_comp3, matutities)}

ani = swap_curve_animation(curves, range(len(p_comp1)), fps = 60, min_maturity = 0, max_maturity = 30, 
                          min_yield = -1, max_yield = 1, title = '250 rolling PCA as of ')
 

plt.show()

In [ ]:
html_str = ani.to_jshtml()

#with open(PATH_HTML + 'Film_PCA.html', 'w') as f:
#    f.write(html_str)

HTML(html_str)
